Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [2]:
%pip install python-dotenv
%pip install requests-oauthlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

In [11]:
TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session

# Cette partie sert à récupérer les ID de connexions qui sont dans un fichier .env que l'on doit créer à chaque fois que l'on copie le repository github, cela permet de les conserver secret
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Requette d'un token pour avoir accès à l'API 
res = requests.post(
  TOKEN_URL,
  data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
  }
)

# Récupération du token 
token = res.json()
client = OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données . La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [13]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

In [23]:
response = client.post(
    API_HOST+"/search", # On ajoute ici l'endpoint souhaité
    json = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "fromAdvancedRecherche": False,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "crimes",
            "operateur": "OU",
            "criteres": [
              {
                "valeur": "délits",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              },
              {
                "valeur": "délinquance",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              }
            ],
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "ET",
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 100,
    "pageNumber": 1,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond": "LODA_DATE",
  "results": {
    "champsRetour": ["titre", "date"]
  }
  }
).json()

In [27]:
results = response.get('results', [])
print(results)


[{'titles': [{'id': 'LEGITEXT000043404965_23-04-2021', 'cid': 'JORFTEXT000043403203', 'title': "LOI n° 2021-478 du 21 avril 2021 visant à protéger les mineurs des <mark>crimes</mark> et délits sexuels et de l'inceste (1)", 'legalStatus': None, 'startDate': None, 'endDate': None, 'nature': None}], 'text': None, 'type': '_doc', 'nature': 'LOI', 'origin': 'LEGI', 'etat': 'VIGUEUR', 'date': '2021-04-23T00:00:00.000+0000', 'sections': [], 'num': None, 'jorfText': None, 'numParution': None, 'datePublication': '2021-04-22T00:00:00.000+0000', 'dossiersLegislatifs': [{'id': 'JORFDOLE000043032513', 'ancienId': None, 'origine': None, 'url': None, 'nature': None, 'titre': "LOI n° 2021-478 du 21 avril 2021 visant à protéger les mineurs des crimes et délits sexuels et de l'inceste", 'type': None, 'dateCreation': None, 'dateDerniereModification': 1619395200000, 'legislature': None, 'echeancier': '', 'dateMajEcheancier': None, 'exposeMotif': None, 'arborescence': None, 'dossiers': [], 'refInjection': 

In [ ]:
client.post(
    API_HOST+"/search", # On ajoute ici l'endpoint souhaité
    json = {
  "recherche": {
    "filtres": [
      {
        "valeurs": [
          "LOI",
          "ORDONNANCE",
          "ARRETE"
        ],
        "facette": "NATURE"
      },
      {
        "dates": {
          "start": "2015-01-01",
          "end": "2018-01-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "sort": "SIGNATURE_DATE_DESC",
    "fromAdvancedRecherche": False,
    "secondSort": "ID",
    "champs": [
      {
        "criteres": [
          {
            "proximite": 2,
            "valeur": "dispositions",
            "criteres": [
              {
                "valeur": "soins",
                "operateur": "ET",
                "typeRecherche": "UN_DES_MOTS"
              },
              {
                "proximite": "3",
                "valeur": "fonction publique",
                "operateur": "ET",
                "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"
              }
            ],
            "operateur": "ET",
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "ET",
        "typeChamp": "TITLE"
      }
    ],
    "pageSize": 10,
    "operateur": "ET",
    "typePagination": "DEFAUT",
    "pageNumber": 1
  },
  "fond": "LODA_DATE"
}
).json()


{'executionTime': 50,
 'results': [{'titles': [{'id': 'LEGITEXT000035603847_01-01-2017',
     'cid': 'JORFTEXT000035600747',
     'title': "Arrêté du 20 septembre 2017 modifiant l'arrêté du 7 janvier 2014 relatif à l'échelonnement indiciaire du corps de directeur des soins de la fonction publique hospitalière et à l'échelonnement indiciaire des emplois fonctionnels de directeur des soins de certains établissements mentionnés à l'article 2 de la loi n° 86-33 du 9 janvier 1986 portant <mark>dispositions</mark> statutaires relatives à la fonction publique hospitalière",
     'legalStatus': None,
     'startDate': None,
     'endDate': None,
     'nature': None}],
   'text': None,
   'type': '_doc',
   'nature': 'ARRETE',
   'origin': 'LEGI',
   'etat': 'VIGUEUR',
   'date': '2017-01-01T00:00:00.000+0000',
   'sections': [],
   'num': None,
   'jorfText': None,
   'numParution': None,
   'datePublication': '2017-09-22T00:00:00.000+0000',
   'dossiersLegislatifs': [],
   'nor': 'SSAH1631818